<a href="https://colab.research.google.com/github/AOMDiputacio/Scrapper/blob/main/Generar_tablas_de_precios.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

A partir del archivo '/content/drive/MyDrive/Scrapper-Pruebadecolchones/url_nombre_a_scrappear.csv'


#Hypnia

In [ ]:
from bs4 import BeautifulSoup
import requests as r
import pandas as pd
import json
from google.colab import drive

# Montar Google Drive
drive.mount('/content/drive', force_remount=True)

# Rutas de los archivos en Google Drive
ruta_archivo_csv = '/content/drive/MyDrive/Scrapper-Pruebadecolchones/url_nombre_a_scrappear.csv'

# Leer el CSV desde Google Drive con especificación de la codificación
df_urls = pd.read_csv(ruta_archivo_csv, encoding='latin1')

# Check for NaN values in the 'URL' column
df_urls = df_urls.dropna(subset=['URL'])

# Filtrar las filas que contienen 'hypnia.es' en la columna 'url'
df_urls = df_urls[df_urls['URL'].str.contains('hypnia.es')]

# Obtener la lista de URLs, nombres, y variantes desde el DataFrame
urles = df_urls['URL'].tolist()
nombres = df_urls['Nombre'].tolist()
urles = [urle + "?variant=" for urle in urles]

# Crear una lista para almacenar los DataFrames de cada URL
dfs = []

# Iterar sobre las URLs
for urle, nombre in zip(urles, nombres):  # Iterate over both URL and nombre simultaneously

    # Función para obtener precios desde la URL
    def obtener_precios(id):
        url = f"{urle}{id}"
        reql = r.get(url)
        soup = BeautifulSoup(reql.text, "html.parser")
        precios = soup.find('div', class_='product_price')

        if precios:
            precio_original = precios.find('s').text if precios.find('s') else None
            precio_con_descuento = precios.find_all('span')[1].text if len(precios.find_all('span')) > 1 else None
            return precio_original, precio_con_descuento
        else:
            return None, None

    # Obtener el contenido de la página
    reql = r.get(urle)
    soup = BeautifulSoup(reql.text, "html.parser")

    # Encontrar todos los scripts con type 'application/json'
    scripts = soup.find_all('script', type='application/json')

    # Iterar sobre los scripts y encontrar el que contiene datos relevantes
    for script in scripts:
        if "name" in script.string:
            schema = script.string
            break

    # Extraer datos del JSON
    data = json.loads(schema)

    # Crear DataFrame
    df = pd.DataFrame(data, columns=["name", "id", "title", "price"])

    # Renombrar columnas
    df.columns = ["name", "id", "title", "price"]

    # Agregar una columna 'nombre' al DataFrame
    df['nombre'] = nombre

    # Iterar sobre cada fila del DataFrame y añadir las columnas
    for index, row in df.iterrows():
        id_variant = row['id']
        precio_original, precio_con_descuento = obtener_precios(id_variant)
        df.at[index, 'precio_original'] = precio_original
        df.at[index, 'precio_con_descuento'] = precio_con_descuento

    # Agregar el DataFrame a la lista
    dfs.append(df)

# Concatenar todos los DataFrames en uno solo
final_df = pd.concat(dfs, ignore_index=True)



# Formatear tabla y columnas

# Definir una función para formatear precios y limpiar título
def formatear_datos(row):
    # Formatear precios
    for columna_precio in ['precio_original', 'precio_con_descuento']:
        if pd.notna(row[columna_precio]):
            # Eliminar el símbolo € y el punto de miles, luego reemplazar la coma decimal por un punto y eliminar espacios
            row[columna_precio] = str(row[columna_precio]).replace('€', '').replace('.', '').replace(' ', '')
            # Agregar el punto decimal para indicar decimales
            row[columna_precio] = row[columna_precio].replace(',', '.', 1)

    # Limpiar título y suprimir espacios
    if ' (cm)' in row['title']:
        row['title'] = row['title'].split(' (cm)')[0].replace(' ', '')

    return row

# Aplicar la función a las filas del DataFrame
final_df = final_df.apply(formatear_datos, axis=1)

#cambiar nombre de columna
final_df.rename(columns={'title': 'dimension'}, inplace=True)

# Mostrar solo las últimas tres columnas del DataFrame
#print(final_df[['nombre', 'label', 'precio_original', 'precio_con_descuento']])


# Crear un nuevo DataFrame con las columnas deseadas
matriz_precios_hypnia = final_df[['nombre', 'dimension', 'precio_original', 'precio_con_descuento']]

# Imprimir el resultado
print(matriz_precios_hypnia)

# Subir resultados en Google drive

# Rutas de los archivos en Google Drive
ruta_archivo_json = '/content/drive/MyDrive/Scrapper-Pruebadecolchones/matriz_hypnia.json'
ruta_archivo_csv = '/content/drive/MyDrive/Scrapper-Pruebadecolchones/matriz_hypnia.csv'

# Guardar el DataFrame en un archivo JSON en Google Drive
#matriz_precios_hypnia.to_json(ruta_archivo_json, orient='records', lines=True)

# Guardar el DataFrame en un archivo CSV en Google Drive
matriz_precios_hypnia.to_csv(ruta_archivo_csv, index=False)


Mounted at /content/drive
                            nombre        dimension precio_original  \
0    Almohada Efecto Plumas Hypnia  40 x 70 cm (x2)           99.00   
1       Almohada Ergonómica Hypnia    56 x 36 cm x2          128.00   
2       Almohada Ergonómica Hypnia       56 x 36 cm           70.00   
3          Almohada Luna de Hypnia            50x70           55.00   
4          Almohada Luna de Hypnia            50x70           99.00   
..                             ...              ...             ...   
149      Topper Esencial de Hypnia          160x200          215.00   
150      Topper Esencial de Hypnia          180x200          239.00   
151       Topper Premium de Hypnia           90x190          239.00   
152       Topper Premium de Hypnia          160x200          375.00   
153       Topper Premium de Hypnia          180x200          425.00   

    precio_con_descuento  
0                  49.00  
1                  89.00  
2                  49.00  
3            

#Pikolin

In [ ]:
from bs4 import BeautifulSoup
import requests as r
import pandas as pd
import json
from google.colab import drive
import re

# Montar Google Drive
drive.mount('/content/drive', force_remount=True)

# Rutas de los archivos en Google Drive
ruta_archivo_csv = '/content/drive/MyDrive/Scrapper-Pruebadecolchones/url_nombre_a_scrappear.csv'

# Leer el CSV desde Google Drive con especificación de la codificación
df_urls = pd.read_csv(ruta_archivo_csv, encoding='latin1')

# Check for NaN values in the 'URL' column
df_urls = df_urls.dropna(subset=['URL'])

# Filtrar las filas que contienen 'pikolin.com' en la columna 'URL'
df_urls_pikolin = df_urls[df_urls['URL'].str.contains('pikolin.com')]

# Inicializar una lista vacía para almacenar los resultados de cada fila
results_list = []

def extract_json_objects(script_tags):
    json_objects = []
    for script_tag in script_tags:
        try:
            if script_tag.string:
                json_content = json.loads(script_tag.string)
                json_objects.append(json_content)

                pattern = r'"defaultProductId": (\d+)'
                match = re.search(pattern, script_tag.string)

                if match:
                    default_product_id = match.group(1)
                    target_product_id = default_product_id

        except json.JSONDecodeError as e:
            print(f"Error al decodificar JSON en la etiqueta de script:\n{script_tag}\nError: {e}")

    return json_objects, target_product_id

def get_product_data(json_content, target_product_id):
    product_id_data = json_content.get(f"[product-id='{target_product_id}']", {}).get("Magento_Swatches/js/swatch-renderer", {}).get("jsonConfig", {}).get("attributes", {})
    return product_id_data

def create_table(data, column_names):
    table = pd.DataFrame(data, columns=column_names)
    return table

# Iterar a través de todas las filas en df_urls_pikolin
for index, row in df_urls_pikolin.iterrows():
    url = row['URL']
    nombre = row['Nombre']

    # Realizar una solicitud para la URL actual
    response = r.get(url)
    soup = BeautifulSoup(response.text, "html.parser")

    # Encontrar todas las etiquetas de script
    script_tags = soup.find_all('script', {'type': 'text/x-magento-init'})

    # Extraer objetos JSON de las etiquetas de script
    json_objects, target_product_id = extract_json_objects(script_tags)

    # Extract JSON objects from script tags
    for script_tag in script_tags:
        try:
            # Check if the 'string' attribute is not None
            if script_tag.string:
                # Check if the script tag contains the target product ID
                if f'"[product-id=\'{target_product_id}\']":' in script_tag.string:
                    # Parse the text content of the script tag as JSON
                    json_content = json.loads(script_tag.string)

        except json.JSONDecodeError as e:
            print(f"Error decoding JSON in script tag:\n{script_tag}\nError: {e}")

    # Obtener datos relacionados con el producto usando el ID del producto
    product_id_data = get_product_data(json_content, target_product_id)

    # Crear una tabla con columnas 'Id', 'label' y 'products'
    column_names1 = ['Id', 'label', 'products']
    table1_data = [{'Id': option['id'], 'label': option['label'], 'products': ', '.join(option['products'])} for option in product_id_data.get('272', {}).get('options', [])]
    df_table1 = create_table(table1_data, column_names1)

    # Obtener datos de precios de opciones
    option_prices_data = json_content.get(f"[product-id='{target_product_id}']", {}).get("Magento_Swatches/js/swatch-renderer", {}).get("jsonConfig", {}).get("optionPrices", {})

    # Crear una tabla con columnas 'Label', 'Old Price' y 'Final Price'
    column_names2 = ['Label', 'Old Price', 'Final Price']
    table2_data = [{'Label': label, 'Old Price': prices['oldPrice']['amount'], 'Final Price': prices['finalPrice']['amount']} for label, prices in option_prices_data.items()]
    df_table2 = create_table(table2_data, column_names2)

    # Fusionar las tablas en columnas clave comunes
    merged_table = pd.merge(df_table1, df_table2, left_on=['products'], right_on=['Label'], how='left')

    # Crear una nueva columna 'nombre' en merged_table con el mismo valor para todas las filas
    merged_table.insert(0, 'nombre', nombre)

    # Agregar la tabla fusionada a la lista de resultados
    results_list.append(merged_table)

# Concatenar todos los resultados en un solo DataFrame
final_result = pd.concat(results_list, ignore_index=True)

# Eliminar las columnas 'label' y 'products'
final_result = final_result.drop(final_result.columns[[1, 3, 4]], axis=1)

# Imprimir el resultado final
matriz_precios_pikolin = final_result

matriz_precios_pikolin.rename(columns={'label': 'dimension', 'Old Price': 'precio_original', 'Final Price': 'precio_con_descuento'}, inplace=True)

print(matriz_precios_pikolin)

# Subir resultados en Google drive

# Montar Google Drive
# drive.mount('/content/drive', force_remount=True)

# Rutas de los archivos en Google Drive
ruta_archivo_json = '/content/drive/MyDrive/Scrapper-Pruebadecolchones/matriz_pikolin.json'
ruta_archivo_csv = '/content/drive/MyDrive/Scrapper-Pruebadecolchones/matriz_pikolin.csv'

# Guardar el DataFrame en un archivo JSON en Google Drive
#final_result.to_json(ruta_archivo_json, orient='records', lines=True)

# Guardar el DataFrame en un archivo CSV en Google Drive
final_result.to_csv(ruta_archivo_csv, index=False)



Mounted at /content/drive
             nombre dimension  precio_original  precio_con_descuento
0      Pikolin City    90x190            733.0                 366.5
1      Pikolin City   105x190            813.0                 406.5
2      Pikolin City   135X190            977.0                 488.5
3      Pikolin City   150X190           1115.0                 557.5
4      Pikolin City   150X200           1227.0                 613.5
..              ...       ...              ...                   ...
64  Pikolin Marisma    90x200           1250.0                 562.5
65  Pikolin Marisma   105x200           1390.0                 625.5
66  Pikolin Marisma   160x190           1840.0                 828.0
67  Pikolin Marisma   135X200           1558.0                 701.1
68  Pikolin Marisma   180X200           2182.0                 981.9

[69 rows x 4 columns]


#Emma


In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import json
from google.colab import drive

# Montar Google Drive
drive.mount('/content/drive', force_remount=True)

# Rutas de los archivos en Google Drive
ruta_archivo_csv = '/content/drive/MyDrive/Scrapper-Pruebadecolchones/url_nombre_a_scrappear.csv'

# Leer el CSV desde Google Drive con especificación de la codificación
df_urls = pd.read_csv(ruta_archivo_csv, encoding='latin1')

# Check for NaN values in the 'URL' column
df_urls = df_urls.dropna(subset=['URL'])

# Filtrar las filas que contienen 'emma-colchon.es' en la columna 'url'
df_urls = df_urls[df_urls['URL'].str.contains('emma-colchon.es')]

# Inicializar una lista para almacenar los DataFrames resultantes
resultados = []

# Iterar sobre cada URL en df_urls
for index, row in df_urls.iterrows():
    url = row['URL']
    nombre = row['Nombre']
    descuento = row['Descuento']

    # Realizar la solicitud HTTP
    response = requests.get(url)

    # Verificar si la solicitud fue exitosa (código 200)
    if response.status_code == 200:
        # Obtener el contenido HTML
        html_content = response.text

        # Crear un objeto BeautifulSoup para analizar el HTML
        soup = BeautifulSoup(html_content, 'html.parser')

        # Encontrar el script con id="structured-data_0"
        target_script = soup.find('script', {'id': 'structured-data_0'})

        # Extraer el contenido del script si se encuentra
        if target_script:
            # Obtener el contenido del script como texto
            script_content = target_script.string

            # Cargar el JSON desde el script
            data = json.loads(script_content)

            # Crear un DataFrame de pandas
            df = pd.DataFrame(data)

            # Seleccionar las columnas 'name' y 'offers'
            df = df[['name', 'offers']]

            # Expandir la columna 'offers' en nuevas columnas
            df = pd.concat([df.drop(['offers'], axis=1), df['offers'].apply(pd.Series)], axis=1)

            # Renombrar columnas'
            df = df.rename(columns={'name': 'lbl'})
            df = df.rename(columns={'price': 'precio_con_descuento'})

            # Seleccionar las columnas finales
            df = df[['lbl', 'precio_con_descuento']]

            # Conservar solo la parte después del último espacio en la columna 'lbl'
            df['lbl'] = df['lbl'].apply(lambda x: x.split(' ')[-1])

            # Añadir las columnas 'nombre' y 'precio original'
            df['nombre'] = nombre
            df['precio_con_descuento'] = pd.to_numeric(df['precio_con_descuento'], errors='coerce')
            descuento_numerico = float(descuento.strip('%')) / 100
            df['precio_original'] = round(df['precio_con_descuento'] / (1 - descuento_numerico), 2) if descuento_numerico != 0 else round(df['price'], 2)

            # Reorganizar las columnas para colocar 'nombre' en la primera posición
            df = df[['nombre', 'lbl', 'precio_original', 'precio_con_descuento']]

            # Agregar el DataFrame resultante a la lista
            resultados.append(df)
        else:
            print(f"No se encontró un script con id='structured-data_0' en la URL: {url}")
    else:
        print(f"Fallo al recuperar la página web. Código de estado: {response.status_code} en la URL: {url}")

# Concatenar todos los DataFrames en uno solo
resultado_final = pd.concat(resultados, ignore_index=True)

# Imprimir el DataFrame final
matriz_precios_emma = resultado_final

matriz_precios_emma.rename(columns={'lbl': 'dimension'}, inplace=True)

print(matriz_precios_emma)

# Rutas de los archivos en Google Drive
ruta_archivo_json = '/content/drive/MyDrive/Scrapper-Pruebadecolchones/matriz_emma.json'
ruta_archivo_csv = '/content/drive/MyDrive/Scrapper-Pruebadecolchones/matriz_emma.csv'

# Guardar el DataFrame en un archivo JSON en Google Drive
# resultado_final.to_json(ruta_archivo_json, orient='records', lines=True)

# Guardar el DataFrame en un archivo CSV en Google Drive
resultado_final.to_csv(ruta_archivo_csv, index=False)


Mounted at /content/drive
                                 nombre dimension  precio_original  \
0              Almohada Microfibra Emma     40x70            37.37   
1   Almohada Viscoelástica Premium Emma     70x40            99.86   
2                    Base Tapizada Emma    90x190           242.71   
3                    Base Tapizada Emma   135x190           342.71   
4                    Base Tapizada Emma   150x190           385.57   
..                                  ...       ...              ...   
67                   Cama Tapizada Emma   135x190           599.87   
68                   Cama Tapizada Emma   140x200           642.73   
69                   Cama Tapizada Emma   140x200           471.30   
70                   Cama Tapizada Emma   150x200           699.87   
71                   Cama Tapizada Emma   150x200           528.44   

    precio_con_descuento  
0                  29.90  
1                  69.90  
2                 169.90  
3                 239.90 

# Juntando todas las tablas

In [ ]:
# Obtén las 4 primeras columnas de cada DataFrame
column_indices = [0, 1, 2, 3]

# Selecciona las columnas deseadas de cada DataFrame
hypnia_selected_columns = matriz_precios_hypnia.iloc[:, column_indices]
pikolin_selected_columns = matriz_precios_pikolin.iloc[:, column_indices]
emma_selected_columns = matriz_precios_emma.iloc[:, column_indices]

# Concatena los DataFrames seleccionados
matriz_precios_scrapping = pd.concat([hypnia_selected_columns, pikolin_selected_columns, emma_selected_columns], ignore_index=True)

matriz_precios_scrapping = matriz_precios_scrapping.iloc[:, column_indices]

# Asigna nombres a las columnas resultantes
matriz_precios_scrapping.columns = ['nombre', 'dimension', 'precio_original', 'precio_con_descuento']

# Imprimir el resultado
print(matriz_precios_scrapping)

# Subir resultados en Google drive

# Rutas de los archivos en Google Drive
ruta_archivo_json = '/content/drive/MyDrive/Scrapper-Pruebadecolchones/matriz_precios_scrapping.json'
ruta_archivo_csv = '/content/drive/MyDrive/Scrapper-Pruebadecolchones/matriz_precios_scrapping.csv'

# Guardar el DataFrame en un archivo JSON en Google Drive
matriz_precios_scrapping.to_json(ruta_archivo_json, orient='records', lines=True)

# Guardar el DataFrame en un archivo CSV en Google Drive
matriz_precios_scrapping.to_csv(ruta_archivo_csv, index=False)

                            nombre        dimension precio_original  \
0    Almohada Efecto Plumas Hypnia  40 x 70 cm (x2)           99.00   
1       Almohada Ergonómica Hypnia    56 x 36 cm x2          128.00   
2       Almohada Ergonómica Hypnia       56 x 36 cm           70.00   
3          Almohada Luna de Hypnia            50x70           55.00   
4          Almohada Luna de Hypnia            50x70           99.00   
..                             ...              ...             ...   
290             Cama Tapizada Emma          135x190          599.87   
291             Cama Tapizada Emma          140x200          642.73   
292             Cama Tapizada Emma          140x200           471.3   
293             Cama Tapizada Emma          150x200          699.87   
294             Cama Tapizada Emma          150x200          528.44   

    precio_con_descuento  
0                  49.00  
1                  89.00  
2                  49.00  
3                  49.00  
4           

#Exportar a Bubble

In [ ]:
import pandas as pd
import requests

# Convertir DataFrame a JSON
json_data = matriz_precios_scrapping.to_json(orient='records')
print(json_data)

# URL de la API de Bubble
api_url = 'https://tu-api-bubble.com/tu-endpoint'

# Configurar encabezados para la solicitud POST (asegúrate de revisar la documentación de la API)
headers = {'Content-Type': 'application/json'}

# Enviar solicitud a la API de Bubble
response = requests.post(api_url, data=json_data, headers=headers)

# Verificar la respuesta de la API
if response.status_code == 200:
    print('Datos enviados exitosamente a la API de Bubble.')
else:
    print(f'Error al enviar datos a la API de Bubble. Código de estado: {response.status_code}')
    print(response.text)  # Imprimir el contenido de la respuesta en caso de error


[{"nombre":"Almohada Efecto Plumas Hypnia","dimension":"40 x 70 cm (x2)","precio_original":"99.00","precio_con_descuento":"49.00"},{"nombre":"Almohada Ergon\u00f3mica Hypnia","dimension":"56 x 36 cm x2","precio_original":"128.00","precio_con_descuento":"89.00"},{"nombre":"Almohada Ergon\u00f3mica Hypnia","dimension":"56 x 36 cm","precio_original":"70.00","precio_con_descuento":"49.00"},{"nombre":"Almohada Luna de Hypnia","dimension":"50x70","precio_original":"55.00","precio_con_descuento":"49.00"},{"nombre":"Almohada Luna de Hypnia","dimension":"50x70","precio_original":"99.00","precio_con_descuento":"89.00"},{"nombre":"Almohada Luna de Hypnia","dimension":"50x70","precio_original":"55.00","precio_con_descuento":"49.00"},{"nombre":"Almohada Luna de Hypnia","dimension":"50x70","precio_original":"99.00","precio_con_descuento":"89.00"},{"nombre":"Base Tapizada Esencial Hypnia","dimension":"90x190","precio_original":"309.00","precio_con_descuento":"199.00"},{"nombre":"Base Tapizada Esencia

ConnectionError: ignored